# Text Classification Using RoBERTa
 Fine Tuning Transformer for MultiLabel Text Classification

Installing Transformers

In [1]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.4 MB/s eta 0:00:00


In [4]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

## Importing Necessary Libraries

In [26]:
# Importing stock ml libraries
import numpy as np
import pandas as pd
from sklearn import metrics
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import transformers
import requests
from io import BytesIO
import tarfile
import torch.optim as optim

from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report




## Extract Data from source

In [6]:
url = "https://s3.amazonaws.com/fast-ai-nlp/amazon_review_full_csv.tgz"
response = requests.get(url)
file = tarfile.open(fileobj=BytesIO(response.content))
file.extractall()

In [7]:
train = pd.read_csv('amazon_review_full_csv/train.csv',usecols=[0,1,2], names=['Rating', 'Title','ReviewText'],nrows=10000)
test = pd.read_csv('amazon_review_full_csv/test.csv',usecols=[0,1,2], names=['Rating', 'Title','ReviewText'],nrows=3500)

In [8]:
train.shape

(10000, 3)

In [9]:
test.shape

(3500, 3)

In [10]:
# Define function to classify ratings
def classify_rating(rating):
    if rating > 3:
        return 'Positive'
    elif rating < 3:
        return 'Negative'
    else:
        return 'Neutral'

In [11]:
train['Sentiment'] = train['Rating'].apply(lambda x: classify_rating(x))
test['Sentiment'] = test['Rating'].apply(lambda x: classify_rating(x))

In [12]:
train,test = train[['Sentiment','ReviewText']], test[['Sentiment','ReviewText']]

## Convert labels to numeric values

In [13]:

label2id = {"Positive": 2, "Neutral": 1, "Negative": 0}
train["label"] = train["Sentiment"].map(label2id)
test["label"] = test["Sentiment"].map(label2id)

<ipython-input-13-fb1943870542>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["label"] = train["Sentiment"].map(label2id)
<ipython-input-13-fb1943870542>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["label"] = test["Sentiment"].map(label2id)


In [14]:
train["Sentiment"] = train["label"]
test["Sentiment"] = test["label"]

<ipython-input-14-f2db7ca2a232>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["Sentiment"] = train["label"]
<ipython-input-14-f2db7ca2a232>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["Sentiment"] = test["label"]


In [15]:
train,test = train[['Sentiment','ReviewText']], test[['Sentiment','ReviewText']]

In [16]:
train.head()

,Sentiment,ReviewText
0,1,Gave this to my dad for a gag gift after direc...
1,2,I hope a lot of people hear this cd. We need m...
2,2,I'm reading a lot of reviews saying that this ...
3,2,The music of Yasunori Misuda is without questi...
4,2,Probably the greatest soundtrack in history! U...


In [17]:
test.head()

,Sentiment,ReviewText
0,0,"This model may be ok for sedentary types, but ..."
1,2,This is a fast read filled with unexpected hum...
2,0,I bought one of these chargers..the instructio...
3,0,I was excited to find a book ostensibly about ...
4,0,"I am a big JVC fan, but I do not like this mod..."


## Load the RoBERTa tokenizer and encode the text:

In [20]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

In [21]:
class AmazonReviewDataset(Dataset):
    def __init__(self, reviews, labels):
        self.reviews = reviews
        self.Sentiment = labels

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        review = str(self.reviews[idx])
        label = self.Sentiment[idx]
        encoding = tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=512,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "label": torch.tensor(label, dtype=torch.long)
        }



In [22]:
train_dataset = AmazonReviewDataset(train["ReviewText"], train["Sentiment"])
val_dataset = AmazonReviewDataset(test["ReviewText"], test["Sentiment"])

## Create data loaders:

In [23]:
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)


## Load the pre-trained BERT model for sequence classification:

In [24]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(label2id))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

## Define the optimizer and learning rate scheduler:

In [27]:
optimizer = optim.RMSprop(model.parameters(), lr=0.001, alpha=0.9, eps=1e-8)
total_steps = len(train_loader) * 5  # 5 epochs
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)


## Train the model:

In [28]:
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs} - Avg Loss: {avg_loss:.4f}")

    # Validation
    model.eval()
    val_predictions = []
    val_labels = []
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            _, predictions = torch.max(logits, dim=1)
            val_predictions.extend(predictions.detach().cpu().numpy())
            val_labels.extend(labels.detach().cpu().numpy())
    val_acc = accuracy_score(val_labels, val_predictions)
    print(f"Validation Accuracy: {val_acc:.4f}")


Epoch 1/5 - Avg Loss: 1.0943
Validation Accuracy: 0.3986
Epoch 2/5 - Avg Loss: 1.0644
Validation Accuracy: 0.3986
Epoch 3/5 - Avg Loss: 1.0648
Validation Accuracy: 0.3986
Epoch 4/5 - Avg Loss: 1.0673
Validation Accuracy: 0.3986
Epoch 5/5 - Avg Loss: 1.0626
Validation Accuracy: 0.3986


## Evaluate the model on training set

In [29]:
# Evaluation on the train set

model.eval()
train_predictions = []
train_labels = []
with torch.no_grad():
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        _, predictions = torch.max(logits, dim=1)
        train_predictions.extend(predictions.detach().cpu().numpy())
        train_labels.extend(labels.detach().cpu().numpy())

train_acc = accuracy_score(train_labels, train_predictions)
classification_rep = classification_report(train_labels, train_predictions, target_names=label2id.keys())

print(f"Train Accuracy: {train_acc:.4f}")
print("\nClassification Report:")
print(classification_rep)



Train Accuracy: 0.4073

Classification Report:
              precision    recall  f1-score   support

    Positive       0.41      1.00      0.58      4073
     Neutral       0.00      0.00      0.00      2010
    Negative       0.00      0.00      0.00      3917

    accuracy                           0.41     10000
   macro avg       0.14      0.33      0.19     10000
weighted avg       0.17      0.41      0.24     10000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
